<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/C%C3%B3pia_de_Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Modules:
Spark, pyspark, Java.

Bellow it is the old fashion way, risky not Recomended

In [1]:
""" 
# # Installing Java
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # Dowload Spark
! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
# # Decompressing archives
! tar xf spark-3.2.1-bin-hadoop2.7.tgz
# # installing findspark
! pip install -q findspark
# # installing pyspark
! pip install pyspark==3.2.1
 """

' \n# # Installing Java\n! apt-get install openjdk-8-jdk-headless -qq > /dev/null\n# # Dowload Spark\n! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz\n# # Decompressing archives\n! tar xf spark-3.2.1-bin-hadoop2.7.tgz\n# # installing findspark\n! pip install -q findspark\n# # installing pyspark\n! pip install pyspark==3.2.1\n '

In [2]:
""" 
# Importing os library
import os
# # Java envirorment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# # Spark envirorment variable
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
 """

' \n# Importing os library\nimport os\n# # Java envirorment variable\nos.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"\n# # Spark envirorment variable\nos.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"\n '

# Better and consistent library that makes the same function as the code above



In [3]:
! pip install pyspark py4j
! pip install pyodbc


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Importing modules and pandas library that will be used later.**

In [4]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
import pyodbc
# import findspark

# findspark.init()

In [5]:
# instatianting spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

Gooogle Colab Version

In [6]:
""" 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 
"""

" \nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True) \n"

# Next cells acquire parquet data from remote files, and transpose it to spark format

In [7]:
fille = r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/BrStocksFormated.parquet"

stocks = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [8]:
fille = r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Ebit.parquet"

ebit = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [9]:
fille = r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Stat.parquet"

stat = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [10]:
fille = r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Price.parquet"

price = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [11]:
fille =r'C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/RAW_DATA/Parquet/Balance.parquet'

balance = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [12]:
# Stocks table
stocks.createOrReplaceTempView("General")
# Ebit table
ebit.createOrReplaceTempView("Ebit")
# Stat table
stat.createOrReplaceTempView("Stat")
# Price table
price.createOrReplaceTempView("Price")
# Balance table
balance.createOrReplaceTempView("Balance")

# Note 1: 
'DetToEquity' must be in percentage, try to get it done here or otherwise in Power Bi

---

# Note 2: 
Find a way to calculate Beta values from 
historical data get it from other sources, but try to calculate it localy (Talk with Felipe, about the period of time that will be used as 'ytd' or montlhy). 

# Fixed:
Beta values could be acquired trough yahoo finance

Listing brazilian companies that will be researched and getting hystorical financial data
filters that our Analyst indicates: Raking companies by: EBIT/EV > 0, ROE > 0, **Daily equity < 1.000.000.000**, revenueGrowth > 0 AND Will be done apart :(5-year-history) and all margins must be postive

In [13]:
# Already called at Calc Table
Gen = spark.sql("""
SELECT * FROM General
WHERE Moeda != 'USD' AND EBITDA != 0 AND ROA != 0 AND ROE != 0;
""").createOrReplaceTempView("Gen")

In [14]:
#Gen.show()

In [15]:
# Already called at RSCH table
Ebit = spark.sql("""
SELECT * FROM Ebit
where Divida_Liquida != 0;
""").createOrReplaceTempView("Debt")

In [16]:
#Ebit.show()

In [17]:
# Already called at RSCH table
Price = spark.sql("""
SELECT * FROM Price
WHERE valor_de_mercado != 0;
""")#.createOrReplaceTempView("Prices")

In [18]:
Price.show()

+--------+----------------+-----------------+-----------------+
|  Ticker|valor_de_mercado|volume_de_mercado|__index_level_0__|
+--------+----------------+-----------------+-----------------+
|ZAMP3.SA|      1290876288|                0|                0|
|AERI3.SA|       964650304|                0|                3|
|DOTZ3.SA|       113162640|                0|                5|
|MLAS3.SA|      1763443968|                0|                8|
|NINJ3.SA|       134109408|                0|               11|
|MODL3.SA|      1179109888|                0|               14|
|VITT3.SA|      1594519040|                0|               15|
|KRSA3.SA|       874195328|                0|               16|
|ALLD3.SA|       470764032|                0|               23|
|ATMP3.SA|        27776526|                0|               25|
|JSLG3.SA|      1982957056|                0|               27|
|ELMD3.SA|      1802294016|                0|               29|
|OPCT3.SA|       617591488|             

Alternative operations, while dealing with a little amount of data, and it can be joined here

In [19]:
Stat = spark.sql("""
SELECT * FROM Stat
WHERE Beta != 0.5;
""")#.createOrReplaceTempView("Stat")

In [20]:
#Stat.show()

In [21]:
CALCULATE = spark.sql("""
SELECT * FROM Gen
WHERE Margem_Bruta > 0.0 AND Margem_EBITIDA > 0.0 AND 
Margem_Operacional > 0.0 AND Margem_liquida > 0.0 AND 
ROE > 0 AND Crescimento_de_Receita_3T > 0;
""")#.createOrReplaceTempView("Calc")

In [22]:
#CALCULATE.show()

In [23]:
RSCH = spark.sql("""
SELECT p.Ticker, 
       p.valor_de_mercado, 
       p.volume_de_mercado, 
       e.EBIT, 
       e.Divida_Liquida 
FROM Price as p, Ebit as e
WHERE (p.valor_de_mercado + e.Divida_Liquida)/e.EBIT > 0.0 AND 
p.Ticker == e.Ticker; 
""")#.createOrReplaceTempView("RSCH")

In [24]:
Balance = spark.sql("""
SELECT  Ticker,
        Data_de_Reporte,
        Moeda,
        Receita_a_Pagar,
        Receitas_a_Receber,
        Bens_de_Capital
FROM Balance
""")

In [25]:
Balance.show()

+---------+-------------------+-----+---------------+------------------+---------------+
|   Ticker|    Data_de_Reporte|Moeda|Receita_a_Pagar|Receitas_a_Receber|Bens_de_Capital|
+---------+-------------------+-----+---------------+------------------+---------------+
|A2MC34.SA|2019-12-30 21:00:00|  USD|        5.433E8|           1.603E8|      1000000.0|
|A2MC34.SA|2020-12-30 21:00:00|  USD|        2.988E8|            2.31E7|      2300000.0|
|A2MC34.SA|2021-12-30 21:00:00|  USD|        3.771E8|            8.54E7|      5100000.0|
| AALR3.SA|2019-12-30 21:00:00|  BRL|        5.667E7|          8.8789E7|      6.12412E8|
| AALR3.SA|2020-12-30 21:00:00|  BRL|       9.4887E7|          7.9679E7|      6.12412E8|
| AALR3.SA|2021-12-30 21:00:00|  BRL|      1.17548E8|         1.04502E8|      6.12412E8|
|AAPL34.SA|2019-09-29 21:00:00|  USD|      4.6236E10|         2.2926E10|      4.5174E10|
|AAPL34.SA|2020-09-29 21:00:00|  USD|      4.2296E10|          1.612E10|      5.0779E10|
|AAPL34.SA|2021-09-29

In [26]:
calendario = spark.sql("""
SELECT 
    MIN(Data_de_Reporte) as Data_Inicio
FROM Balance 
""")

In [27]:
calendario.show()

+-------------------+
|        Data_Inicio|
+-------------------+
|2019-03-30 21:00:00|
+-------------------+



ev = número total de papeis x cotação + divida liquida
ev = market value +  liquid debt 

PSR = PREÇO DA AÇÃO / RECEITA LIQUIDA POR AÇÃO 

Pandas setting DataFrame to pandas format:

In [28]:
df_stat = Stat.toPandas()

df_calc = CALCULATE.toPandas()

df_rsc = RSCH.toPandas()

df_balance = Balance.toPandas()

df_calendario = calendario.toPandas()

c:\Users\video\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
c:\Users\video\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [29]:
df_calendario

,Data_Inicio
0,2019-03-30 21:00:00


Excel format tables

In [30]:
""" 
df_stat.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Excel/Stat.xlsx",
                 sheet_name="stat")

df_calc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Calcl.xlsx",
                 sheet_name="calc")

df_rsc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/RSCH.xlsx",
                sheet_name="research")

df_balance.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Balance.xlsx",
                sheet_name="balance")
"""

' \ndf_stat.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Excel/Stat.xlsx",\n                 sheet_name="stat")\n\ndf_calc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Calcl.xlsx",\n                 sheet_name="calc")\n\ndf_rsc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/RSCH.xlsx",\n                sheet_name="research")\n\ndf_balance.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Balance.xlsx",\n                sheet_name="balance")\n'

Parquet Format tables

In [31]:
df_balance.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Balance.parquet")

df_stat.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calcl.parquet")

df_rsc.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/RSCH.parquet")

df_calc.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calcl.parquet")

df_calendario.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calendario.parquet")

Local Connection to SQL Server

In [32]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-7VLQ9AP\SQLEXPRESS;'
                      'Database=FeA;'
                      'Trusted_Connection=yes;'
                      )

cursor = conn.cursor()

In [33]:
str(df_calendario.columns).replace("'","")

'Index([Data_Inicio], dtype=object)'

In [37]:
for index, linha in df_calendario.iterrows():
    print(index,linha)

0 Data_Inicio   2019-03-30 21:00:00
Name: 0, dtype: datetime64[ns]


Commiting to SQL SERVER - Calendario

In [36]:
for index, linha in df_calendario.iterrows():
        cursor.execute("Insert into Calendario(Data_Inicio) values(?)",
                       linha.Data_Inicio)

cursor.commit()

Commiting to SQL SERVER - Balance

In [38]:
for index, linha in df_balance.iterrows():
    print(index,linha)

0 Ticker                          A2MC34.SA
Data_de_Reporte       2019-12-30 21:00:00
Moeda                                 USD
Receita_a_Pagar               543300000.0
Receitas_a_Receber            160300000.0
Bens_de_Capital                 1000000.0
Name: 0, dtype: object
1 Ticker                          A2MC34.SA
Data_de_Reporte       2020-12-30 21:00:00
Moeda                                 USD
Receita_a_Pagar               298800000.0
Receitas_a_Receber             23100000.0
Bens_de_Capital                 2300000.0
Name: 1, dtype: object
2 Ticker                          A2MC34.SA
Data_de_Reporte       2021-12-30 21:00:00
Moeda                                 USD
Receita_a_Pagar               377100000.0
Receitas_a_Receber             85400000.0
Bens_de_Capital                 5100000.0
Name: 2, dtype: object
3 Ticker                           AALR3.SA
Data_de_Reporte       2019-12-30 21:00:00
Moeda                                 BRL
Receita_a_Pagar                56670000.0

In [39]:
str(df_balance.columns).replace("'","")

'Index([Ticker, Data_de_Reporte, Moeda, Receita_a_Pagar,\n       Receitas_a_Receber, Bens_de_Capital],\n      dtype=object)'

In [40]:
cursor.open()
for index, linha in df_balance.iterrows():
        cursor.execute("Insert into Balance(Ticker, Data_de_Reporte, Moeda, Receita_a_Pagar,Bens_de_Capital) values(?,?,?,?,?)",
                       linha.Ticker,linha.Data_de_Reporte, linha.Moeda, linha.Receita_a_Pagar,linha.Bens_de_Capital)

cursor.commit()
cursor.close()

ProgrammingError: Attempt to use a closed cursor.

Cloud Version

In [ ]:
""" 
server_name = "jdbc:sqlserver://DESKTOP-7VLQ9AP\SQLEXPRESS"
database_name = "FeA"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "Balance"
username = "SQL_ADM"
password = "and010500" # Please specify password here

try:
  df_balance.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", table_name) \
    .option("user", username) \
    .option("password", password) \
    .save()

except ValueError as error :
    print("Connector write failed", error)
 """